In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Mon Feb 13 16:58:15 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    50W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 25.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 KB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 20.2 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=ca2e6bed21e4afe250e4da9c4065f6023907a494e91bbcb7d25896e373fe23f6
  Stored in directory: /root/.cache/pip/wheels/4c/8e/7e/72fbc243e1aeecae64a96875432e70d4e92f3d2d18123be004
Successfully built pathtools
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
!pip install pyproj

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.8 MB/s eta 0:00:00


In [ ]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/PHD_2022/SAC/')  #Root folder
sys.path.insert(0,'/content/drive/MyDrive/PHD_2022/SAC/utils/') # Libraries folder

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pickle as pkl

from utils import map_vis_without_lanelet

# EXPERIMENT PARAMETERS
in_timesteps = 10
out_timesteps = 10
in_features = 5
out_features = 2

max_cars_per_cluster = 6

# Run name
wandb_project_name = "ablation-intersection"

train_dataset_path = "/content/drive/MyDrive/PHD_2022/SAC/data/INTERACTION-Dataset-DR-single-v1_2/train/20-m/20-m-6-c-train.csv"
test_dataset_path = '/content/drive/MyDrive/PHD_2022/SAC/data/INTERACTION-Dataset-DR-single-v1_2/train/20-m/20-m-6-c-test.csv'

scaler_save_path = '/content/drive/MyDrive/PHD_2022/SAC/models/ablation/graph-50-m-6-c-011/scaler.pkl'
scaler_reload_path = '/content/drive/MyDrive/PHD_2022/SAC/models/ablation/graph-50-m-6-c-011/scaler.pkl'

model_save_path = '/content/drive/MyDrive/PHD_2022/SAC/models/ablation/graph-50-m-6-c-011/'
model_reload_path = '/content/drive/MyDrive/PHD_2022/SAC/models/ablation/graph-50-m-6-c-011/'

csv_out_path_train = '/content/drive/MyDrive/PHD_2022/SAC/results/results.csv'
csv_out_path_test = '/content/drive/MyDrive/PHD_2022/SAC/results/results.csv'

map_path = "/content/drive/MyDrive/PHD_2022/SAC/data/INTERACTION-Dataset-DR-single-v1_2/maps/DR_USA_Intersection_MA.osm"

#history_save_path = 'models/graph50-3c/LSTM3-graph-50-5c_history.pkl'



**METRICS**

In [ ]:
#@title
# ADE function (single)

import math
def average_displacement_error(pred,ground):

    e = []

    for p,g in zip(pred,ground):
        e.append( math.sqrt (((p[0] - g[0]) ** 2) + ((p[1] - g[1]) ** 2)))

    return np.sum(e)/len(e)

# ADE function over BATCH (mean)

def calculate_error_batch(results, ground):

    e = [] 

    for r, g in zip(results, ground):
        e.append(average_displacement_error(r,g))
    
    return np.sum(e)/len(e)

In [ ]:
import wandb
from wandb.keras import WandbCallback

wandb.init(project=wandb_project_name)

#wandb
wandb_callback = WandbCallback(monitor='val_loss', mode='min', log_gradients=False,save_model=False)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


**START**

In [ ]:
# Load clustered dataset
dataframe = pd.read_csv(train_dataset_path, sep=',')
# This is the train dataframe without normalization, it will be needed after to write the results to disk .csv
dataframe_raw = dataframe   

In [ ]:
# Scaling
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler


def scale_full_dataframe(dataframe, scaler_reload_path):

    dataframe_scaled = dataframe[["x", "y", "vx", "vy","psi_rad"]]
    dataframe = dataframe.drop(columns=["x", "y", "vx", "vy","psi_rad"], axis=1) # Keep only meta-data features

    if scaler_reload_path is not None: # if this is test set
        scaler = pd.read_pickle(scaler_reload_path)
    else:
        scaler = MinMaxScaler()

    dataframe_scaled = pd.DataFrame(scaler.fit_transform(dataframe_scaled), columns=["x", "y", "vx", "vy","psi_rad"])

    dataframe = pd.concat([dataframe, dataframe_scaled], axis=1)
    
    if scaler_reload_path is None:  # if this is training set save scaler to file
        # Save scaler to file
        
        with open(scaler_save_path, "wb") as outfile:
            pkl.dump(scaler, outfile)

    return dataframe

In [ ]:
# Not sending a pre-calculated scaler because this is training set
# But if you are reloading for evaluating over test send you should send the pre-computed scaler
                                                                                                                                                                        
dataframe = scale_full_dataframe(dataframe, scaler_reload_path = None)  

In [ ]:
def split_groups(dataframe):

    case_id = dataframe.groupby(["case_id", "label"])

    scenary_x = []
    scenary_y = []


    for ix,element in case_id:
        l = element.groupby(["track_id"])
    
        agents = [] # n trajectories (scenary-(aka: cluster)), one per agent

        # Get all scenary-(aka:cluster) trajectories
        for ig, eg in l:
            feat = eg[["x", "y", "vx", "vy","psi_rad"]]
            f_list = feat.values.tolist()
            agents.append(f_list)
    
        # separate each trajectory

        vehicle_temp_x = [] # Needed to store all vehicles and after fill empty spaces to reach 14 max vehicles per cluster
        vehicle_temp_y = []

        vehicle_count = 0

        for ix,tj in enumerate(agents):
            temp = np.array(tj)
            temp_x = temp[0:in_timesteps,:]  # 0:10,:
            temp_y = temp[in_timesteps:,0:out_features]  # 10:, 0:2 only predict x,y

            if( ix == 0):
                d_x = temp_x 
                d_y = temp_y
            else:  
                d_x = np.hstack((d_x,temp_x))                
                d_y = np.hstack((d_y,temp_y))
                

            vehicle_count = vehicle_count + 1
        
    
        d_x = d_x.reshape(in_timesteps,vehicle_count,in_features) # in_timesteps=10, 1= only one car, in_features = 5 features
        # Add missing vehicles the model needs a squared matrix with 14 vehicles as maximum
        if(vehicle_count < max_cars_per_cluster):
            missing_vehicles = np.zeros((in_timesteps,max_cars_per_cluster-vehicle_count,in_features))
            d_x = np.concatenate((d_x, missing_vehicles), axis=1)

    
        d_y = d_y.reshape(out_timesteps,vehicle_count,out_features) # out_timesteps=10, 1=only one car, out_features = 2 features
        # Add missing vehicles the model needs a squared matrix with 14 vehicles as maximum
        if(vehicle_count < max_cars_per_cluster):
           missing_vehicles = np.zeros((out_timesteps,max_cars_per_cluster-vehicle_count,out_features))
           d_y = np.concatenate((d_y, missing_vehicles), axis=1)


        scenary_x.append(d_x)
        scenary_y.append(d_y)   

    return np.array(scenary_x),np.array(scenary_y)

In [ ]:
scene_x_train,scene_y_train = split_groups(dataframe) # unfold groups

# Print training shape
print(scene_x_train.shape)
print(scene_y_train.shape)

(768, 10, 6, 5)
(768, 10, 6, 2)


In [ ]:
# Store interaction groups to use in the future
#scene_x_train_interaction = scene_x_train
#scene_y_train_interaction = scene_y_train

In [ ]:
import typing

#adjacency_matrix = np.array([[1, 1, 1], [1, 1, 1],[1, 1, 1],[1, 1, 1],[1, 1, 1]])
#adjacency_matrix = np.full((14, 14), 1.0)

adjacency_matrix = np.array( [[ 1,  1,  1, 1, 1, 1], 
                              [ 1,  1,  1, 1, 1, 1],
                              [ 1,  1,  1, 1, 1, 1],
                              [ 1,  1,  1, 1, 1, 1],
                              [ 1,  1,  1, 1, 1, 1],
                              [ 1,  1,  1, 1, 1, 1]
                              ])


#adjacency_matrix = np.array( [[ 1,  1,  1,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0],
#                              [ 1,  1,  1,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0],
#                              [ 1,  1,  1,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0],
#                              [ 1,  1,  1,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0],
#                              [ 1,  1,  1,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0],
#                              [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
#                              [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
#                              [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
#                              [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
#                              [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
#                              [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
#                              [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
#                              [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
#                              [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
#                              ])


class GraphInfo:
    def __init__(self, edges: typing.Tuple[list, list], num_nodes: int):
        self.edges = edges
        self.num_nodes = num_nodes


node_indices, neighbor_indices = np.where(adjacency_matrix == 1)

graph = GraphInfo(
    edges=(node_indices.tolist(), neighbor_indices.tolist()),
    num_nodes=adjacency_matrix.shape[0],
)
print(f"number of nodes: {graph.num_nodes}, number of edges: {len(graph.edges[0])}")

number of nodes: 6, number of edges: 36


In [ ]:
#@title
# Display All Intersection
import random

def print_all_intersection(prev_list,ground_list,prediction_list,n,ix):
    
    prev = prev_list[ix:ix+n]
    ground = ground_list[ix:ix+n]

    if prediction_list is not None:
        pred = prediction_list[ix:ix+n]
   
    fig, ax = plt.subplots(figsize=(20,20))


    for i,p in enumerate(prev):
        
        g = ground[i]

        p = tf.transpose(p,[1,0]) 
        g = tf.transpose(g,[1,0])

        ax.plot(p[0], p[1],'-o',color='gray',linestyle='dashed')
        ax.plot(g[0], g[1],'-o',color='green',linestyle='dashed')

        # to print all lines of the dataset

        #colors = ['blue','green','red','cyan','magenta','yellow','black']
        #random_color = random.randint(0, 6)

        #ax.plot(p[0], p[1],'-',color=colors[random_color],linestyle='solid')
        #ax.plot(g[0], g[1],'-',color=colors[random_color],linestyle='solid')

        # if you want to print predictions and ground truths at the same time
        if prediction_list is not None:
            r = pred[i]
            r = tf.transpose(r,[1,0])
            ax.plot(r[0], r[1],'-o',color='red',linestyle='dashed')
        
    
    plt.show()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

"""
## Network architecture

Our model for forecasting over the graph consists of a graph convolution
layer and a LSTM layer.
"""

"""
### Graph convolution layer

Our implementation of the graph convolution layer resembles the implementation
in [this Keras example](https://keras.io/examples/graph/gnn_citations/). Note that
in that example input to the layer is a 2D tensor of shape `(num_nodes,in_feat)`
but in our example the input to the layer is a 4D tensor of shape
`(num_nodes, batch_size, input_seq_length, in_feat)`. The graph convolution layer
performs the following steps:

- The nodes' representations are computed in `self.compute_nodes_representation()`
by multiplying the input features by `self.weight`
- The aggregated neighbors' messages are computed in `self.compute_aggregated_messages()`
by first aggregating the neighbors' representations and then multiplying the results by
`self.weight`
- The final output of the layer is computed in `self.update()` by combining the nodes
representations and the neighbors' aggregated messages
"""


class GraphConv(layers.Layer):
    def __init__(
        self,
        in_feat,
        out_feat,
        graph_info: GraphInfo,
        aggregation_type="mean",
        combination_type="concat",
        activation: typing.Optional[str] = None,
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.in_feat = in_feat
        self.out_feat = out_feat
        self.graph_info = graph_info
        self.aggregation_type = aggregation_type
        self.combination_type = combination_type
        self.weight = tf.Variable(
            initial_value=keras.initializers.glorot_uniform()(
                shape=(in_feat, out_feat), dtype="float32"
            ),
            trainable=True,
        )
        self.activation = layers.Activation(activation)

    def aggregate(self, neighbour_representations: tf.Tensor):
        aggregation_func = {
            "sum": tf.math.unsorted_segment_sum,
            "mean": tf.math.unsorted_segment_mean,
            "max": tf.math.unsorted_segment_max,
        }.get(self.aggregation_type)

        if aggregation_func:
            return aggregation_func(
                neighbour_representations,
                self.graph_info.edges[0],
                num_segments=self.graph_info.num_nodes,
            )

        raise ValueError(f"Invalid aggregation type: {self.aggregation_type}")

    def compute_nodes_representation(self, features: tf.Tensor):
        """Computes each node's representation.

        The nodes' representations are obtained by multiplying the features tensor with
        `self.weight`. Note that
        `self.weight` has shape `(in_feat, out_feat)`.

        Args:
            features: Tensor of shape `(num_nodes, batch_size, input_seq_len, in_feat)`

        Returns:
            A tensor of shape `(num_nodes, batch_size, input_seq_len, out_feat)`
        """
        return tf.matmul(features, self.weight)

    def compute_aggregated_messages(self, features: tf.Tensor):
        neighbour_representations = tf.gather(features, self.graph_info.edges[1])
        aggregated_messages = self.aggregate(neighbour_representations)
        return tf.matmul(aggregated_messages, self.weight)

    def update(self, nodes_representation: tf.Tensor, aggregated_messages: tf.Tensor):
        if self.combination_type == "concat":
            h = tf.concat([nodes_representation, aggregated_messages], axis=-1)
        elif self.combination_type == "add":
            h = nodes_representation + aggregated_messages
        else:
            raise ValueError(f"Invalid combination type: {self.combination_type}.")

        return self.activation(h)

    def call(self, features: tf.Tensor):
        """Forward pass.

        Args:
            features: tensor of shape `(num_nodes, batch_size, input_seq_len, in_feat)`

        Returns:
            A tensor of shape `(num_nodes, batch_size, input_seq_len, out_feat)`
        """
        nodes_representation = self.compute_nodes_representation(features)
        aggregated_messages = self.compute_aggregated_messages(features)
        return self.update(nodes_representation, aggregated_messages)


"""
### LSTM plus graph convolution

By applying the graph convolution layer to the input tensor, we get another tensor
containing the nodes' representations over time (another 4D tensor). For each time
step, a node's representation is informed by the information from its neighbors.

To make good forecasts, however, we need not only information from the neighbors
but also we need to process the information over time. To this end, we can pass each
node's tensor through a recurrent layer. The `LSTMGC` layer below, first applies
a graph convolution layer to the inputs and then passes the results through a
`LSTM` layer.
"""


class LSTMGC(layers.Layer):
    """Layer comprising a convolution layer followed by LSTM and dense layers."""

    def __init__(
        self,
        in_feat,
        out_feat,
#        lstm_units: int,
        input_seq_len: int,
        output_seq_len: int,
        graph_info: GraphInfo,
        graph_conv_params: typing.Optional[dict] = None,
        **kwargs,
    ):
        super().__init__(**kwargs)

        # graph conv layer
        if graph_conv_params is None:
            graph_conv_params = {
                "aggregation_type": "mean",
                "combination_type": "concat",
                "activation": None,
            }

        self.graph_conv = GraphConv(in_feat, out_feat, graph_info, **graph_conv_params)

        # SEQ2SEQ BASELINE

        #self.encoder = layers.LSTM(2048, return_sequences=False)
        #self.latent_vector = layers.RepeatVector(10) # RepeatVector = 10 = out_timesteps  !Attention
        #self.decoder = layers.LSTM(2048, return_sequences=True)
        #self.decoder_last = layers.TimeDistributed(layers.Dense(units=2, activation='linear'))  # Prediction 2 output features x,y
        
        # SEQ2SEQ 2 BASELINE
        self.encoder = layers.GRU(1024, return_sequences=True)
        self.encoder_2 = layers.GRU(1024, return_sequences=True)
        self.encoder_3 = layers.GRU(1024, return_sequences=False)
        #self.encoder_3 = layers.LSTM(1024, return_sequences=False)

        self.latent_vector = layers.RepeatVector(10) # RepeatVector = 10 = out_timesteps  !Attention

        self.decoder =   layers.GRU(512, return_sequences=True)
        self.decoder_2 = layers.GRU(512, return_sequences=True)

        #self.decoder_3 = layers.LSTM(512, return_sequences=True)

        self.decoder_last = layers.TimeDistributed(layers.Dense(units=2, activation='linear'))  # Prediction 2 output features x,y

        self.input_seq_len, self.output_seq_len = input_seq_len, output_seq_len

    def call(self, inputs):
        """Forward pass.

        Args:
            inputs: tf.Tensor of shape `(batch_size, input_seq_len, num_nodes, in_feat)`

        Returns:
            A tensor of shape `(batch_size, output_seq_len, num_nodes)`.
        """

        # convert shape to  (num_nodes, batch_size, input_seq_len, in_feat)
        inputs = tf.transpose(inputs, [2, 0, 1, 3])

        gcn_out = self.graph_conv(
            inputs
        )  # gcn_out has shape: (num_nodes, batch_size, input_seq_len, out_feat)
        
        shape = tf.shape(gcn_out)
        num_nodes, batch_size, input_seq_len, out_feat = (
            shape[0],
            shape[1],
            shape[2],
            shape[3],
        )                                    
      
        # LSTM takes only 3D tensors as input
        gcn_out = tf.reshape(gcn_out, (batch_size * num_nodes, input_seq_len, out_feat))
        
        # BASELINE SEQ 2 SEQ
        #encoder_output = self.encoder(gcn_out)
        #latent_vector_output = self.latent_vector(encoder_output)
        #decoder_output = self.decoder(latent_vector_output)
        #decoder_last_output = self.decoder_last(decoder_output) 


        encoder_output = self.encoder(gcn_out)
        encoder_2_output = self.encoder_2(encoder_output)
        encoder_3_output = self.encoder_3(encoder_2_output)

        latent_vector_output = self.latent_vector(encoder_3_output)

        decoder_output = self.decoder(latent_vector_output)
        decoder_2_output = self.decoder_2(decoder_output)
#        decoder_3_output = self.decoder_3(decoder_2_output)

        decoder_last_output = self.decoder_last(decoder_2_output) 

        
        output = tf.reshape(decoder_last_output, (num_nodes, batch_size, self.output_seq_len,2)) # out_feat = 2 features !!!

        return tf.transpose(output, [1, 2, 0, 3 ])
        


"""
## Model training
"""



multi_horizon = False




graph_conv_params = {
    "aggregation_type": "mean",
    "combination_type": "concat",
    "activation": None,
}

# Match library parameters name with my own parameters naming convention
st_gcn = LSTMGC(
    in_features,
    out_features,
    in_timesteps,
    out_timesteps,
    graph,
    graph_conv_params,
)


inputs = layers.Input((in_timesteps, graph.num_nodes, in_features))
outputs = st_gcn(inputs)

model = keras.models.Model(inputs, outputs)



In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),   #original =  0.0000001
    loss=['mse'],
    metrics=['mse', 'mae']
)


In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10, 6, 5)]        0         
                                                                 
 lstmgc (LSTMGC)             (None, 10, 6, 2)          19698700  
                                                                 
Total params: 19,698,700
Trainable params: 19,698,700
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# !!!IMPORTANT!!! only uncomment and run this line to load a pre-trained model file for re-training or evaluating 
#model = tf.keras.models.load_model(model_reload_path)

In [ ]:
# early stopping   # 0.00000000000000001
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta = 0.00000000000001, patience=30, mode='min', restore_best_weights=True) #0.00005

# callbacks
callbacks_list = [wandb_callback,early_stopping_callback]

In [ ]:
from sklearn.utils import shuffle
model.fit(
    scene_x_train, scene_y_train, 
    validation_split=0.2,
    batch_size = 64, #original 64 , mycode = 128
    epochs=4000,  #original development 1000 , mycode = 10000
    shuffle = True,
    callbacks=callbacks_list,
)

Epoch 1/4000
10/10 [==============================] - 15s 200ms/step - loss: 0.0935 - mse: 0.0935 - mae: 0.2469 - val_loss: 0.0564 - val_mse: 0.0564 - val_mae: 0.1910
Epoch 2/4000
10/10 [==============================] - 0s 25ms/step - loss: 0.0473 - mse: 0.0473 - mae: 0.1737 - val_loss: 0.0306 - val_mse: 0.0306 - val_mae: 0.1385
Epoch 3/4000
10/10 [==============================] - 0s 24ms/step - loss: 0.0330 - mse: 0.0330 - mae: 0.1410 - val_loss: 0.0302 - val_mse: 0.0302 - val_mae: 0.1426
Epoch 4/4000
10/10 [==============================] - 0s 23ms/step - loss: 0.0277 - mse: 0.0277 - mae: 0.1323 - val_loss: 0.0205 - val_mse: 0.0205 - val_mae: 0.1121
Epoch 5/4000
10/10 [==============================] - 0s 24ms/step - loss: 0.0229 - mse: 0.0229 - mae: 0.1213 - val_loss: 0.0190 - val_mse: 0.0190 - val_mae: 0.1093
Epoch 6/4000
10/10 [==============================] - 0s 23ms/step - loss: 0.0210 - mse: 0.0210 - mae: 0.1168 - val_loss: 0.0177 - val_mse: 0.0177 - val_mae: 0.1063
Epoch 7/

In [ ]:
# save the model in tensorflow format
model.save(model_save_path)